In [5]:
import os
from collections import defaultdict
import cv2
import numpy as np

In [6]:
# Dictionary to hold features for each person
person_features = defaultdict(list)

# Use ORB as the feature extraction technique
orb = cv2.ORB_create()

# Load person features from face_data folder
def load_person_features():
    face_data = "face_data"
    for person_name in os.listdir(face_data):
        person_folder = os.path.join(face_data, person_name)
        if os.path.isdir(person_folder):
            for img_name in os.listdir(person_folder):
                img_path = os.path.join(person_folder, img_name)
                image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if image is not None:
                    keypoints, descriptors = orb.detectAndCompute(image, None)
                    if descriptors is not None:
                        person_features[person_name].append(descriptors)
    
    # Print number of feature sets per person
    for person, features in person_features.items():
        print(f"{person}: {len(features)} feature sets found.")

In [7]:
# Function to recognize a person from a frame
def recognize_person(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = orb.detectAndCompute(gray, None)
    
    if descriptors is not None:
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        best_match_count = 0
        recognized_person = "Unknown"
        
        for person, feature_sets in person_features.items():
            for features in feature_sets:
                try:
                    matches = bf.match(descriptors, features)
                    matches = sorted(matches, key=lambda x: x.distance)
                    
                    if len(matches) > best_match_count:
                        best_match_count = len(matches)
                        recognized_person = person
                except:
                    continue
        
        return recognized_person
    
    return "No face detected"

In [8]:
def main():
    # Load training data
    load_person_features()
    
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    
    window_name = 'Face Recognition'
    cv2.namedWindow(window_name)
    
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break
        
        # Recognize person in the frame
        person_name = recognize_person(frame)
        
        # Display the person's name on the frame
        cv2.putText(frame, person_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        # Display the resulting frame
        cv2.imshow(window_name, frame)
        
        # Break the loop with 'q' key or if window is closed
        if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
            break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # When done, release the capture
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Benison: 10 feature sets found.
Harsh: 10 feature sets found.
Nandalal: 10 feature sets found.
